## Coursework

 Machine Learning- COIY065H7  
 William Gilpin  
 wgilpi01@dcs.bbk.ac.uk  
 wgilpin@gmail.com  
 
<br/>
<br/>
<br/>
<br/>
<br/>
 
 
 

## Introduction

This task is undertaken as courework for the Birkbeck MSc Data Analytics pathway.
The task is to assess the performance of the WAME optimiser [Mosca 2017],
"which involve adapting a different
learning rate for each weight rather than using a single, global, learning
rate for the entire network, we are able to reach close to state–of–the–art
performance on the same architectures, and improve the training time and
accuracy"

## Methodology and Design

The approach is to compare WAME with 3 other optimisation methods, as originally referennced in the WAME paper [Mosca 2017].
These are

- Stochastic Gradient Descent (SGD) with momentum
- RMSprop [Tieleman 2012]
- Adam [Kingma 2014]

### WAME Optimizer

WAME (weight-wise adaptive learning rates with moving average estimator)
modifies the learning rates *per weight* to improve training performance for a given time.

As with Rprop [Riedmiller 1993], WAME takes into account the sign of the gradient and not its magnitude. In a given iteration,

$
   \zeta_{ij}(t) =
\begin{cases}
    \min(\zeta_{ij}(t-1)\times \eta_+, \zeta_{max}),& \text{if } \partial\gt 0\\
    \max(\zeta_{ij}(t-1)\times \eta_+, \zeta_{max}),& \text{if } \partial\lt 0\\
    \zeta_{ij}(t-1)              & \text{otherwise}
\end{cases}
$

The full algorithm also includes the following hyperparameters:
- an exponential decay factor, $\alpha$
- a maximum per-weight acceleration factor for when the gradient is positive (clipping): $\zeta_{max}$
- a mimimum per-weight acceleration factor value for when the gradient is negative (clipping): $\zeta_{min}$
- an acceleration hyperparamter for when the gradient is positive: $\eta_+$
- an acceleration hyperparamter for when the gradient is negative: $\eta_-$

#### The algorithm

1. **pick** $\alpha, \eta_+, \eta_-, \zeta_{min}, \zeta_{max}$
2. $\theta_{ij}(0) = 0, Z_{ij} (0) = 0, \zeta_{ij} = 1|\forall i, j$
3. **for all** $t \in [1..T]$ **do**
4. $\quad$
    **if** $\frac{\partial E(t)}{\partial w_{ij}}\cdot \frac{\partial E(t-1)}{\partial w_{ij}}>0$
    **then**
5. $\quad \quad \zeta_{ij}(t) =\min(\zeta_{ij}(t-1)\times \eta_+, \zeta_{max})$
6. $\quad$
    **else if** $\frac{\partial E(t)}{\partial w_{ij}}\cdot
    \frac{\partial E(t-1)}{\partial w_{ij}}>0$ **then**
7. $\quad \quad \zeta_{ij}(t) =\max(\zeta_{ij}(t-1)\times \eta_+, \zeta_{min})$
8. $\quad$ **end if**
9. $\quad Z_{ij} (t) = \alpha Z_{ij} (t−1) + (1 − \alpha)\zeta_{ij}(t)$
10. $\quad \theta_{ij} (t) = \alpha \theta_{ij} (t−1) + (1 − \alpha){(\frac{\partial E(t)}{\partial w_{ij}}(t))}^2$
11. $\quad \Delta w_{ij}(t)= -\lambda Z_{ij}\frac{\partial E(t)}{\partial w_{ij}}\frac{1}{\theta_{}ij(t)}$
12. $\quad w_{ij}(t+1) = w_{ij}(t)+\Delta w_{ij}(t)$
13. **end for**

<div align="center"><strong>WAME Algorithm</strong></div>

To understand WAME it is useful to consider Rprop optimisation. Rprop deals with the fact that there is wide variation in the magnitude of the gradient on different weights. Compared to previous algorithms which used a global learning rate, Rprop maintains a per-weight learning rate. The algorithm doesn't use the magnitude of the gradietnt, rather it uses the sign of the gradient and whether the sign has changed. If the sign is unchanged between rounds then the direction of change is correct, so we accelerate by mulitplying the step size by say $\eta_+ = 1.2$. If the sign has changed then we have overshot the optinal value, so we reduce the step size by multiplying it by say $\eta_- = 0.1$. This helps reduce the convergence to local minima.

WAME is similar to Rprop in that it applies a per weight accceleration factor $\zeta_{ij}$, but to overcome difficulties Rprop has with large datasets and mini-batch approaches, WAME also uses the Exponential Moving Average (EMA) of $\zeta_{ij}$ and an exponential decay factor, $\alpha$.



### Data

The dataset used is the Statlog (Landsat Satellite) Data Set [Srinivasan 1993]. This dataset contains small ($3\times{3}$)
multispectral images along with their classifications:

1. red soil
2. cotton crop
3. grey soil
4. damp grey soil
5. soil with vegetation stubble
6. mixture class (all types present)
7. very damp grey soil

Each record contains 3 sets of 3x3 pixel values, each set being for a different frequency, and a classification.

The raw data allows 7 classes, but only 6 have samples, so the missign class is removed leaving 6 classes.

The data is loaded and the classes converted into 1-hot encoding for training the multinomial logistic regression. 1-hot encoding is the translation od a set of categories into a matrix where for a given row only a single column value is set to 1, all the others being zero. This way a column represents the class in the original data.

```
classes = [A,C,B,B,C,A]

one_hot_classes = A | B | C
                  ---------
                  1   0   0
                  0   0   1
                  0   1   0
                  0   1   0
                  0   0   1
                  1   0   0
```

### 1 Model

#### 1.1 Approach
The problem is treated as a multinomial logistic regression, assigning one of 6 possible output classes to
each sample.

The model trained is a dense sequential ANN using Keras. The model structure has three hidden dense layers:

 Layer (type)      |           Output Shape   |           Param # | Activation
 -----------------|----------------------------|----------------|----
 input_1 (InputLayer)      | (None, 36)    |            0     |
dense_1 (Dense)         |   (None, 64)       |         2368  | Relu
dense_2 (Dense)         |   (None, 128)      |         8320  | Relu
dropout_1 (Dropout)     |   (None, 128)      |         0 |
dense_3 (Dense)         |   (None, 64)       |         8256 | sigmoid
dense_4 (Dense)         |   (None, 6)        |         390  | softmax

Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0

The loss function is Keras' built-in `categorical_crossentropy` and the optimizer varies per experiment.

This model structure was iterated over the following parameters to determine optimal performance:
- Number of hidden layers: 1-3
- Units per layer: 20-128
- Dropout layers: 0-2
- Dropout fraction: 0.1-0.5
- Dense activation functions: relu/sigmoid

Various tuning functions were created to iterate over these parameters, as well as hand tuning.

The final layer softmax selects the model class with the highest probability as the chosen class.

*Callbacks*

Two keras callbacks were created:
- a CSV logger which logs loss for each epoch duting training
- a custom timer callback which records the time at each epoch

#### 1.2 Techniques and parameters
The loss function is Keras' built-in `categorical_crossentropy` and the optimizer varies per experiment.

This model structure was iterated over the following parameters to determine optimal performance:
- Number of hidden layers: 1-3
- Units per layer: 20-128
- Dropout layers: 0-2
- Dropout fraction: 0.1-0.5
- Dense activation functions: relu/sigmoid

Various tuning functions were created to iterate over these parameters, as well as hand tuning.

*Regularisation*

L2 regularisation was used to reduce the risk of overfitting. Without regularisation
a model penalises for error (loss), but in order to minimise overfitting the regularised model will
also penalise for model complexity. The final model has a large number of units per layer, and regularisation
improved performance.

Without Regularisation  
$\quad minimise(Error(y,\hat{y}))$

With Regularisation  
$\quad minimise(Error(y,\hat{y}) + Complexity(model))$

L2 regularisation allows for the complexity by adding a factor to the error based on the sum of the
squares of the weights in the model, on the understanding that a lot of larger-magnitude weights
probably implies a complex model.

L2 regularisation  
$\quad minimise(Error(y,\hat{y}) + \lambda\sum_{i}{w_i}^2)$

*Parameters*

The experiment started with default parameters for all models. Refinement led to the following choices:

WAME: Epsilon = 0.01 (tested $10^{-1} \rightarrow 10^{-5}$)  
Adam: Learning rate = 0.01  
RMSprop: Learning rate=0.01

Epochs: 300 (tested 20-300)

Batch Size: 64 (tested 40-128)

L2 regulariser: Lambda=0.01 (tested 0.1, 0.01, 0.001)


## 2. Experiments

The experimental approach followed these steps:

- Data pre-processing
- Model iterations deriving parameters & structure as above
- Model execution per optimiser collecting:
  - execution times per epoch
  - scores per 60 epochs
  
#### 2.1 Findings
  
*Loss*

Adam performed well, with minimum loss of 0.866 and predictable improvement as the numbert of iterations increased.
![Adam Scores](figures/Adam-score.png)

SGD was less predictable. Its lowest loss was 0.581 after 180 epochs, but the loss then increased suggesting overfitting after 180 epochs.

![SGD Scores](figures/SGD-score.png)

RMSprop minimum loss was 1.775 after 300 iterations, and apparently still improving.

![RMSprop Scores](figures/RMSprop-score.png)

WAME performed worse than the other 3 optimisers, achieving a minimum loss of 3.180 after 300 epochs. Although loss was still reducing at this point, the loss is till much higher than other methods.

![WAME Scores](figures/WAME-score.png)

*Times*

Suprisingly, WAME was the slowest method (see Conclusions), performing slower than the others under all conditions.

![Window](figures/all-optimizers-window.png "Time over a slding window")

The times for different numbers of iterations are as follows:
    
N          |Adam       |WAME        |SGD    |RMSprop
----|------------|-----------|-----------|---------
60  | 10.772892  |11.859741  |10.785231  |10.219459
120  |19.247254  |24.450373  |18.957702  |19.192914
180  |28.320045  |34.688663  |27.547161  |28.534963
240  |41.103819  |46.036945  |36.807399  |37.486725
300  |53.155509  |57.753125  |46.042100  |46.922082

![Durations](figures/all-durations.png)


#### 2.2 Discussion

WAME is designed to "increased learning speed without
compromising generalisation performance." [Mosca 2017]. The results presented here do not support that conclusion, leading to doubt about the implementation presented here.

Overall WAME scored worse, and performed slower, than the other methods assessed in all experimental conditions given the model design and the dataset used.

Possible reasons

1. WAME performs badly. In the balance of probabilities the peer-reviewed Mosca paper is more likely to reflect a well implemented algorithm than this implementation, although the lack of access to the Mosca source code makes direct comparison hard.
2. WAME is inappropriate to the problem presented. Although the data is a series of images, those images are themselves 3x3 pixels, which did not allow the use of a convolutional network.
3. The model structure used does not take advantage of WAME.
4. Coding errors. The code can be found at [https://github.com/wgilpin/msc-ml-cw]



## 3. Conclusion

#### 3.1 Summary

Overall the recommendation must be not to adopt WAME, given the results of this analysis. The second recommendation is to improve the analysis, perhaps running WAME against the datasets and models described in the [Mosca 2017] to confirm the approach.

#### 3.2 Areas for improvement

1. Validation of this WAME implementation: [Mosca 2017] evaluates WAME against MNIST & CIFAR datasets, utilising CNNs. Test this implementation against the same data using the same models.
2. Consider adding early-stopping. Although the model as run did not converge, with more epochs it should.

## 4. Bibliography

Mosca, A., & Magoulas, G. D. (2017, April).
  Training convolutional networks with weight-wise adaptive learning rates. In ESANN.

Kingma, D. P., & Ba, J. (2014).
  Adam: A method for stochastic optimization. arXiv preprint arXiv:1412.6980.

Riedmiller, M., & Braun, H. (1993, March).
 A direct adaptive method for faster backpropagation learning: The RPROP algorithm. In IEEE international conference on neural networks (pp. 586-591). IEEE.

Srinivasan, A. (1993, February).
  UCI Machine Learning Repository [http://archive.ics.uci.edu/ml/datasets/Statlog+(Landsat+Satellite)].
  Glasgow, UK: University of Strathclyde, Department of Statistics and Data Modeling.

Tieleman, T. & Hinton, G. (2012) Lecture 6.5—RmsProp: Divide the gradient by
a running average of its recent magnitude. COURSERA: Neural Networks for
Machine Learning.


## 5. Academic Declaration

The following sources were adapted for use in this experiment:

Landsat data extraction: https://github.com/abarthakur/trepan_python/blob/master/run.py

WAME algorithm implementation: https://github.com/nitbix/keras-oldfork/blob/master/keras/optimizers.py

Timer callback: from https://stackoverflow.com/a/43186440

I acknowledge that I have read and understood the sections on plagiarism in the College Policy on 
assessment offences and confirm that the work is my own, with the work of others clearly 
acknowledged. I also give my permission to submit my files to the plagiarism testing database 
that the College is using and test it using plagiarism detection software (Turnitin), search 
engines or meta-searching software.

<img src="img/signature.jpg" width="150">




